[![Abrir con Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uxue-sudupe/API-Eustat/blob/main/code_examples/es/tutorial_R_es.ipynb)

# **Tutorial de la API de Eustat con R**

El [banco de datos de Eustat](https://www.eustat.eus/banku/indexArbol.html) contiene una gran cantidad de datos sobre la sociedad, la economía y la población de la Comunidad Autónoma de Euskadi.

Los datos son públicos y accesibles por distintos medios, entre ellos la API, [documentada aquí](https://github.com/uxue-sudupe/API-Eustat/blob/main/README.md). La API facilita enormemente el acceso y la reutilización de los datos. Sin embargo, a menos que se tenga experiencia previa en el acceso a API, su uso puede resultar complejo para un investigador, analista o estudiante.

Este tutorial ofrece una guía rápida sobre cómo acceder a los datos de Eustat a través de su API y descargar su contenido en formato JSON (JavaScript Object Notation) u otro formato permitido. Se muestran ejemplos de cómo descargar los datos y su posterior procesado para obtener datasets que pueden ser tratados con R.

Es necesario utilizar algunos paquetes muy extendidos como `dplyr` o `httr`, y el paquete `rjstat`, específico para tratar con R datos en formato json-stat.

In [15]:
install.packages(c("httr", "jsonlite", "tidyr", "dplyr", "rjstat"), quiet = TRUE)

library(httr)
library(jsonlite)
library(tidyr)
library(dplyr)
library(rjstat)

# **1. Obtener el Catálogo de datos Eustat (listado de tablas)**

Es posible acceder al listado completo de los conjuntos de datos publicados por Eustat, con detalle de código identificador, título y fecha de última actualización.

Las tablas son las que se encuentran en el banco de datos de Eustat: <https://www.eustat.eus/banku/indexArbol.html>

Una llamada a la API se compone de una dirección web principal, una llamada a una función y un conjunto de parámetros. La dirección web principal del banco de datos es

`https://www.eustat.eus/bankupx/api/v1/[lang]/DB`

En este caso, la llamada a la API se hace con la función `GET`. El único parámetro necesario para obtener el catálogo, es el idioma `(lang)`. La respuesta se recibe en formato JSON.

In [16]:

# URL del punto de acceso (endpoint) del catálogo de datos de Eustat

DB <- GET("https://www.eustat.eus/bankupx/api/v1/es/DB")
Catálogo <- fromJSON(rawToChar(DB$content))
head(Catálogo)


,id,type,text,updated
,<chr>,<chr>,<chr>,<chr>
1,PX__fe_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbitos territoriales y sexo. 1997 - 2022,2023-01-09T15:04:41
2,PX__feinem_inem06.px,t,Paro registrado de la C.A. de Euskadi por ámbitos territoriales y sexo,2021-02-17T10:04:22
3,PX__fepycl_pc02cm.px,t,Índice general de precios al consumo (IPC) por Territorio Histórico de la C.A. de Euskadi por índice y mes. Base 2016 (desde 2002),2021-02-16T11:15:00
4,PX_0_fe_pc02cm.px,t,Índice general de precios al consumo (IPC) por Territorio Histórico de la C.A. de Euskadi por índice y mes. Base 2016 (desde 2002),2022-02-15T10:31:14
5,PX_0_fe_pc02m.px,t,Índice general de precios al consumo (IPC) de la C.A. de Euskadi y del Estado por índice y mes. Base 2016 (desde 2002),2022-02-15T10:31:14
6,PX_0_fepycl_pc02cm.px,t,Índice general de precios al consumo (IPC) por Territorio Histórico de la C.A. de Euskadi por índice y mes. Base 2016 (desde 2002),2021-09-14T08:41:11


**Idiomas**

Se pueden obtener los mismos datos en euskera o en inglés. Solo hay que modificar la dirección de la *url* en la parte que contiene el idioma, `www.eustat.eus/bankupx/api/v1/[idioma]/DB`, y reemplazar *es* por *eu* o *en*. Por ejemplo, en euskera sería así:

In [17]:

DB <- GET("https://www.eustat.eus/bankupx/api/v1/eu/DB")
Katalogoa <- fromJSON(rawToChar(DB$content))
head(Katalogoa)


,id,type,text,updated
,<chr>,<chr>,<chr>,<chr>
1,PX__fe_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurralde eremuaren eta sexuaren arabera. 1997 - 2022",2023-01-09T15:04:41
2,PX__feinem_inem06.px,t,"Euskal AEko erregistratutako langabezia, lurralde eremuaren eta sexuaren arabera",2021-02-17T10:04:22
3,PX__fepycl_pc02cm.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), Lurralde Historikoaren, indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2021-02-16T11:15:00
4,PX_0_fe_pc02cm.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), Lurralde Historikoaren, indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2022-02-15T10:31:14
5,PX_0_fe_pc02m.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2022-02-15T10:31:14
6,PX_0_fepycl_pc02cm.px,t,"Euskal AEko eta Estatuko kontsumorako prezioen indize orokorra (KPI), Lurralde Historikoaren, indizearen eta hilaren arabera. Oinarria 2016 (2002tik)",2021-09-14T08:41:11


# **2. Obtener los metadatos de una tabla**

Los metadatos de una tabla contienen información sobre el título de la tabla, cuáles son las dimensiones/variables de desagregación que aparecen en la tabla y los valores que toman. Los metadatos se guardan como código o como texto descriptivo.

Para acceder a los metadatos de una tabla concreta, debemos añadirle a la url del banco de datos el código identificador "id" de la tabla de interés. La dirección tendrá este aspecto: `www.eustat.eus/bankupx/api/v1/[lang]/DB/[id]`

Accedemos, por ejemplo, a los metadatos de la tabla que contiene los datos de "Población estimada de la C.A. de Euskadi por territorio histórico, sexo y año de nacimiento", que se encuentra en esta dirección:

<https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010154_cepv1_ep17.px>

In [18]:

metadatos<- GET("https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010154_cepv1_ep17.px")
metadatos_población <- fromJSON(rawToChar(metadatos$content))
metadatos_población

,code,text,values,valueTexts,time
,<chr>,<chr>,<list>,<list>,<lgl>
1,territorio histórico,territorio histórico,"00, 01, 48, 20","C.A. de Euskadi, Araba/Álava , Bizkaia , Gipuzkoa",NA
2,sexo,sexo,"10, 20, 30","Total , Hombres, Mujeres",NA
3,año de nacimiento,año de nacimiento,"1000, 1109, 1110, 1120, 5800, 5810, 5820, 5830, 5840, 5850, 5860, 5870, 5880, 5890, 5900, 5910, 5920, 5930, 5940, 5950, 5960, 5970, 5980, 5990, 6000, 6010, 6020, 6030, 6040, 6050, 6060, 6070, 6080, 6090, 6100, 6110, 6120, 6130, 6140, 6150, 6160, 6170, 6180, 6190, 6200, 6210, 6220, 6230, 6240, 6250, 6260, 6270, 6280, 6290, 6300, 6310, 6320, 6330, 6340, 6350, 6360, 6370, 6380, 6390, 6400, 6410, 6420, 6430, 6440, 6450, 6460, 6470, 6480, 6490, 6500, 6510, 6520, 6530, 6540, 6550, 6560, 6570, 6580, 6590, 6600, 6610, 6620, 6630, 6640, 6650, 6660, 6670, 6680, 6690, 6700, 6710, 6720, 6730, 6740, 6750, 6760, 6770, 6780, 6790, 6800, 6810, 6820, 6830, 6840, 6850, 6860, 6870, 6880, 6890, 6900, 6910, 6920, 6930, 6940, 6950, 6960, 6970, 6980, 6990, 7000, 7010, 7020, 7030, 7040, 7050, 7060, 7070, 7080, 7090, 7100, 7110, 7120, 7130, 7140, 7150, 7160, 7170, 7180, 7190, 7200, 7210, 7220, 7230, 7240, 7250","Total , 2023 , 2022 , 2021 , 2020 , 2019 , 2018 , 2017 , 2016 , 2015 , 2014 , 2013 , 2012 , 2011 , 2010 , 2009 , 2008 , 2007 , 2006 , 2005 , 2004 , 2003 , 2002 , 2001 , 2000 , 1999 , 1998 , 1997 , 1996 , 1995 , 1994 , 1993 , 1992 , 1991 , 1990 , 1989 , 1988 , 1987 , 1986 , 1985 , 1984 , 1983 , 1982 , 1981 , 1980 , 1979 , 1978 , 1977 , 1976 , 1975 , 1974 , 1973 , 1972 , 1971 , 1970 , 1969 , 1968 , 1967 , 1966 , 1965 , 1964 , 1963 , 1962 , 1961 , 1960 , 1959 , 1958 , 1957 , 1956 , 1955 , 1954 , 1953 , 1952 , 1951 , 1950 , 1949 , 1948 , 1947 , 1946 , 1945 , 1944 , 1943 , 1942 , 1941 , 1940 , 1939 , 1938 , 1937 , 1936 , 1935 , 1934 , 1933 , 1932 , 1931 , 1930 , 1929 , 1928 , 1927 , 1926 , 1925 , 1924 , 1923 , 1922 , 1921 , 1920 , 1919 , 1918 , 1917 , 1916 , 1915 , 1914 , 1913 , 1912 , 1911 , 1910 , 1909 , 1908 , 1907 , 1906 , 1905 , 1904 , 1903 , 1902 , 1901 , 1900 , 1899 , 1898 , 1897 , 1896 , 1895 , 1894 , 1893 , 1892 , 1891 , 1890 , 1889 , 1888 , 1887 , 1886 , 1885 , 1884 , 1883 , 1882 , 1881 , 1880 , 1879 , 1878 , 1877 , 1876 , <= 1875",NA
4,periodo,periodo,"19760101, 19770101, 19780101, 19790101, 19800101, 19810101, 19820101, 19830101, 19840101, 19850101, 19860101, 19870101, 19880101, 19890101, 19900101, 19910101, 19920101, 19930101, 19940101, 19950101, 19960101, 19970101, 19980101, 19990101, 20000101, 20010101, 20020101, 20030101, 20040101, 20050101, 20060101, 20070101, 20080101, 20090101, 20100101, 20110101, 20120101, 20130101, 20140101, 20150101, 20160101, 20170101, 20180101, 20190101, 20200101, 20210101, 20220101, 20220701, 20230101, 20230701, 20240101","1976/01/01, 1977/01/01, 1978/01/01, 1979/01/01, 1980/01/01, 1981/01/01, 1982/01/01, 1983/01/01, 1984/01/01, 1985/01/01, 1986/01/01, 1987/01/01, 1988/01/01, 1989/01/01, 1990/01/01, 1991/01/01, 1992/01/01, 1993/01/01, 1994/01/01, 1995/01/01, 1996/01/01, 1997/01/01, 1998/01/01, 1999/01/01, 2000/01/01, 2001/01/01, 2002/01/01, 2003/01/01, 2004/01/01, 2005/01/01, 2006/01/01, 2007/01/01, 2008/01/01, 2009/01/01, 2010/01/01, 2011/01/01, 2012/01/01, 2013/01/01, 2014/01/01, 2015/01/01, 2016/01/01, 2017/01/01, 2018/01/01, 2019/01/01, 2020/01/01, 2021/01/01, 2022/01/01, 2022/07/01, 2023/01/01, 2023/07/01, 2024/01/01",TRUE


Los metadatos contienen el título de la tabla, junto con los códigos y literales de las variables y los valores. Puede resultar útil generar una tabla en la que se muestren las variables junto a sus valores, incluyendo la correspondencia entre códigos y etiquetas descriptivas.

In [19]:
variables <- metadatos_población$variables

variables_poblacion <- variables %>%
  mutate(
    values = lapply(values, unlist),
    valueTexts = lapply(valueTexts, unlist)
  ) %>%
  unnest(c(values, valueTexts))

variables_poblacion

code,text,values,valueTexts,time
<chr>,<chr>,<chr>,<chr>,<lgl>
territorio histórico,territorio histórico,00,C.A. de Euskadi,NA
territorio histórico,territorio histórico,01,Araba/Álava,NA
territorio histórico,territorio histórico,48,Bizkaia,NA
territorio histórico,territorio histórico,20,Gipuzkoa,NA
sexo,sexo,10,Total,NA
sexo,sexo,20,Hombres,NA
sexo,sexo,30,Mujeres,NA
año de nacimiento,año de nacimiento,1000,Total,NA
año de nacimiento,año de nacimiento,1109,2023,NA


# **3. Obtener los datos de una tabla**

**Hacer una consulta a una tabla para obtener todos los datos**

Para disponer de los datos de las tablas, debemos realizar una consulta a la misma dirección del banco de datos con la función POST. Utilizamos el módulo `rjstat` para trabajar con los datos que vamos a descargar en formato json-stat. En este ejemplo, consultamos todos los datos de la tabla del Producto interior bruto (PIB) de la C.A. de Euskadi (oferta) por territorio histórico, rama de actividad (A-38), tipo de dato y de medida.


In [20]:

# URL del punto de acceso (endpoint) de la tabla

url <- "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_170112_cpib_pib01d.px"

# Cuerpo de la petición: sin filtros (toda la tabla). Por defecto, el resultado se obtiene en formato JSON-stat v.1.2
consulta <- '{"query":[]}'

d.tmp <- POST(url , body = consulta, encode = "json", verbose())
tabla <- fromJSONstat(content(d.tmp, "text"), naming = "label", use_factors = FALSE, silent = FALSE)
PIB <- tabla[[1]]
head(PIB)

,territorio histórico,sector,tipo de dato,tipo de medida,periodo,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1995,449513
2,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1996,439283
3,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1997,466735
4,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1998,564307
5,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),1999,605399
6,C.A. de Euskadi,"1. Agricultura, ganadería y pesca",Nivel,Precios corrientes (miles euros),2000,514139


**Obtener los datos de una tabla, con códigos**

Es posible que nos interese disponer de estos datos con los códigos de las variables, y no con las etiquetas descriptivas o literales. En ese caso, solicitamos que los nombres se capturen como código `(naming="id")` y no como texto `(naming="label")`.

In [21]:
tabla <- fromJSONstat(content(d.tmp, "text"), naming = "id", use_factors = FALSE, silent = FALSE)
PIB_codigo <- tabla[[1]]  # Nos interesa solo la primera lista de la tabla
head(PIB_codigo)

,territorio histórico,sector,tipo de dato,tipo de medida,periodo,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,00,01,10,10,1995,449513
2,00,01,10,10,1996,439283
3,00,01,10,10,1997,466735
4,00,01,10,10,1998,564307
5,00,01,10,10,1999,605399
6,00,01,10,10,2000,514139


**Obtener los datos de una tabla, con una consulta de datos**

Solicitar demasiados datos para luego tener que desechar la mitad es desperdiciar espacio y recursos. Por ello, cuando nos interesa disponer de los datos de una selección de valores, y no todo el contenido de las tablas, debemos realizar una consulta seleccionando las variables y valores de interés.

Para definir la consulta, es necesario conocer el nombre/código de las variables y los valores, que es la información que hemos recibido en el punto anterior, **2. Obtener los metadatos de una tabla.**

Pero para facilitar este paso, la propia web de Eustat dispone de una interfaz de selección de variables y valores para cada tabla, donde podremos seleccionar los datos que nos interesan fácilmente, y obtener la consulta en formato JSON.

Por ejemplo, si queremos datos de la tasa de paro por sexo, iríamos a la página en la que se seleccionan las variables para esta tabla del banco de datos:

<https://www.eustat.eus/bankupx/pxweb/es/DB/-/PX_050403_cpra_tab01.px>

Y hacemos la selección que nos interesa: seleccionamos los datos de la tasa de paro para la C.A. de Euskadi, los 2 sexos (y el total) y los datos trimestrales de los 3 últimos años.

***Ejemplo de Selección***

![](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Paro_seleccion.png)

Tras hacer la selección pulsamos "Continuar" y "Disponer de esta tabla en su aplicación". En este apartado aparecerán tanto la url a la que hay que hacer la petición como la consulta json, que será la que incluiremos en nuestro código.

![](https://raw.githubusercontent.com/uxue-sudupe/API-Eustat/main/img/Paro_consulta.png)

In [22]:
url <- "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_050403_cpra_tab01.px"
consulta <- '{"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"] }},
        {"code": "territorio histórico","selection": {"filter": "item", "values": [ "00"]}},
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50" ]}},
        {"code": "periodo","selection": {"filter": "item","values": ["2023", "2024","2025" ]}}],
         "response": {"format": "json-stat"}} '

d.tmp <- POST(url , body = consulta, encode = "json", verbose())
tabla <- fromJSONstat(content(d.tmp, "text"), naming = "label", use_factors = FALSE, silent = FALSE)
Paro <- tabla[[1]]
head(Paro)

,tasa (%),territorio histórico,sexo,trimestre,periodo,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2023,8.9
2,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2024,7.9
3,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2025,7.0
4,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2023,7.6
5,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2024,7.5
6,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2025,NA


**Modificaciones a la consulta**

Cuando se realiza `"filter": "item"` sobre una variable, se seleccionan los valores que se quieren filtrar (uno o varios).

Si se seleccionan todos los valores, no se está filtrando sobre esa variable, y no aparecerá en la consulta. Tiene el mismo efecto que eliminar la linea que empieza por `{"code":"variable"....}` como haremos ahora con la línea de código de la variable "territorio histórico".

También podemos usar el filtro `"top"`, para seleccionar unicamente un número determinado con los últimos valores de la variable. Este filtro `"top"` se utiliza principalmente con la variable "periodo", "año" o similares.

In [23]:

url = "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_050403_cpra_tab01.px"

query = '{"query": [{"code": "tasa (%)","selection": {"filter": "item", "values": ["30"] }},
    #   {"code": "territorio histórico","selection": {"filter": "*", "values": [ "*"]}},    si eliminamos esta línea, no filtramos la variable "territorio histórico"
        {"code": "trimestre","selection": {"filter": "item","values": ["20", "30", "40","50" ]}},
        {"code": "periodo","selection": {"filter": "top","values": ["2"]}}],             #  filtro "top", últimos 2 valores de la variable "periodo"
         "response": {"format": "json-stat"}}'

d.tmp <- POST(url , body = consulta, encode = "json", verbose())
tabla <- fromJSONstat(content(d.tmp, "text"), naming = "label", use_factors = FALSE, silent = FALSE)
Paro <- tabla[[1]]
head(Paro)


,tasa (%),territorio histórico,sexo,trimestre,periodo,value
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2023,8.9
2,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2024,7.9
3,Tasa de paro,C.A. de Euskadi,Total,Trimestre 1,2025,7.0
4,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2023,7.6
5,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2024,7.5
6,Tasa de paro,C.A. de Euskadi,Total,Trimestre 2,2025,NA


# **Otros formatos de salida**

La salida por defecto de las peticiones API a los datos de las tablas es JSON-stat (versión 1.2). El formato JSON es el más común en APIs, permite un tratamiento flexible de los datos, adecuado para procesamiento automático en scripts (R, Python, etc.), y evita ambigüedades que pueden presentarse en archivos planos como CSV.

También es posible descargar los datos pedidos por API en formatos como csv y excel. Estas salidas pueden ser más adecuadas con usuarios finales no técnicos, en procesos en los que se necesitan csv o excel como entrada (creación de dashboards desde csv/excel) o cuando el objetivo no es el análisis sino facilitar el acceso al público de la datos descargados por API.

A continuación se muestran las salidas csv (3 tipos) y excel, y se guarda una copia de cada una.

In [24]:
# Salida en CSV - texto plano

url <- "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
consulta <- '{"query": [], "response": {"format": "csv"}}'

respuesta_csv <- POST(url, body = consulta)
writeBin(content(respuesta_csv, "raw"), "lengua_materna.csv")


In [25]:
# Salida en CSV2 - formato "pivot-friendly", compatible con tablas dinámicas

url <- "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
consulta <- '{"query": [], "response": {"format": "csv2"}}'

respuesta_csv2 <- POST(url, body = consulta, encode = "raw", content_type_json())

writeBin(content(respuesta_csv2, "raw"), "lengua_materna_csv2.csv")


In [26]:
# Salida en CSV3 - igual que CSV2, pero con códigos en lugar de texto

url <- "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
consulta <- '{"query": [], "response": {"format": "csv3"}}'

respuesta_csv3 <- POST(url, body = consulta, encode = "raw", content_type_json())

writeBin(content(respuesta_csv3, "raw"), "lengua_materna_csv3.csv")


In [27]:
# Salida en Excel

url <- "https://www.eustat.eus/bankupx/api/v1/es/DB/PX_010901_cecv_ni11_1.px"
consulta <- '{"query": [], "response": {"format": "xlsx"}}'

respuesta_xlsx <- POST(url, body = consulta, encode = "raw", content_type_json())

writeBin(content(respuesta_xlsx, "raw"), "lengua_materna.xlsx")
